In [2]:
import numpy as np
import tensorflow as tf

# embedding lookup sparse

https://www.jianshu.com/p/f54eb9715609

https://zhuanlan.zhihu.com/p/94212544

tf.nn.embedding_lookup_sparse(
    params,
    sp_ids,
    sp_weights,
    partition_strategy='mod',
    name=None,
    combiner=None,
    max_norm=None
)

In [5]:
### embedding matrix
example = np.arange(24).reshape(6, 4).astype(np.float32)
embedding = tf.Variable(example)
print(embedding)
print(example)

<tf.Variable 'Variable_2:0' shape=(6, 4) dtype=float32_ref>
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]
 [16. 17. 18. 19.]
 [20. 21. 22. 23.]]


In [6]:
### embedding lookup SparseTensor 定义稀疏矩阵
idx = tf.SparseTensor(indices=[[0, 0], [0, 1], [1, 1], [1, 2], [2, 0]],
                      values=[0, 1, 2, 3, 0], dense_shape=[3, 3])
# 这个稀疏矩阵写成普通形式这样
#---------------------------------------------------------------------#
#array([[0, 1, None],
#       [None, 2, 3],
#       [0, None, None]]) # 为了与0元素相区别，没有填充的部分写成了None
#---------------------------------------------------------------------#
idx

In [16]:
embed = tf.nn.embedding_lookup_sparse(embedding, idx, None, combiner='sum')
sess = tf.Session()
sess.run(tf.global_variables_initializer())
emb = sess.run(embed)
print(embed)
print(emb.shape)
print(emb)


Tensor("embedding_lookup_sparse_1:0", shape=(?, 4), dtype=float32)
(3, 4)
[[ 4.  6.  8. 10.]
 [20. 22. 24. 26.]
 [ 0.  1.  2.  3.]]


结果的shape=(idx.shape[0], embedding.shape[1])，其中结果的第一行等于“embeding的第一行加上embedding的第二行，也就是idx的第一行非None的元素的value，对应了embedding的行数，然后这些行相加“；结果第二行为”embedding第3行和第四行相加“；结果第三行也同理。


# embedding lookup

https://zhuanlan.zhihu.com/p/93846683

tf.nn.embedding_lookup(
    params,
    ids,
    partition_strategy='mod',
    name=None,
    validate_indices=True,
    max_norm=None
)

In [21]:
params = tf.constant([[0.1, 0.4, 0.5, 7.0, 6.4, 1.2, 0.5, 0.3, 3.3, 2.0],
                      [0.3, 0.4, 0.9, 0.8, 0.5, 0.3, 0.7, 0.5, 0.8, 3.2],
                      [0.4, 0.9, 1.1, 4.3, 3.4, 0.2, 0.3, 0.2, 0.5, 0.1]],name='params')
ids = tf.constant([[0, 2, 0, 1], [0, 0, 1, 1]],name='ids')
ids_vec = tf.constant([[0], [2], [0], [1]],name='ids_vec')
with tf.Session() as sess:
    lookup = sess.run(tf.nn.embedding_lookup(params, ids))
    vec_lookup = sess.run(tf.nn.embedding_lookup(params, ids_vec))


print(params)
print(ids)
print(lookup.shape)
print(lookup)
print(ids_vec)
print(vec_lookup.shape)
print(vec_lookup)

Tensor("params_6:0", shape=(3, 10), dtype=float32)
Tensor("ids_5:0", shape=(2, 4), dtype=int32)
(2, 4, 10)
[[[0.1 0.4 0.5 7.  6.4 1.2 0.5 0.3 3.3 2. ]
  [0.4 0.9 1.1 4.3 3.4 0.2 0.3 0.2 0.5 0.1]
  [0.1 0.4 0.5 7.  6.4 1.2 0.5 0.3 3.3 2. ]
  [0.3 0.4 0.9 0.8 0.5 0.3 0.7 0.5 0.8 3.2]]

 [[0.1 0.4 0.5 7.  6.4 1.2 0.5 0.3 3.3 2. ]
  [0.1 0.4 0.5 7.  6.4 1.2 0.5 0.3 3.3 2. ]
  [0.3 0.4 0.9 0.8 0.5 0.3 0.7 0.5 0.8 3.2]
  [0.3 0.4 0.9 0.8 0.5 0.3 0.7 0.5 0.8 3.2]]]
Tensor("ids_vec_4:0", shape=(4, 1), dtype=int32)
(4, 1, 10)
[[[0.1 0.4 0.5 7.  6.4 1.2 0.5 0.3 3.3 2. ]]

 [[0.4 0.9 1.1 4.3 3.4 0.2 0.3 0.2 0.5 0.1]]

 [[0.1 0.4 0.5 7.  6.4 1.2 0.5 0.3 3.3 2. ]]

 [[0.3 0.4 0.9 0.8 0.5 0.3 0.7 0.5 0.8 3.2]]]


# splite

In [22]:
e = np.arange(60).reshape(3,4,5).astype(np.float32)  # [bs, feat_num, emb_dim]  feat_num=m
emb = tf.Variable(e)

In [24]:
split_tensor0 = tf.split(emb, 5 * [1], 2)
split_tensor0

[<tf.Tensor 'split_1:0' shape=(3, 4, 1) dtype=float32>,
 <tf.Tensor 'split_1:1' shape=(3, 4, 1) dtype=float32>,
 <tf.Tensor 'split_1:2' shape=(3, 4, 1) dtype=float32>,
 <tf.Tensor 'split_1:3' shape=(3, 4, 1) dtype=float32>,
 <tf.Tensor 'split_1:4' shape=(3, 4, 1) dtype=float32>]

In [43]:
m = tf.matmul(split_tensor0,split_tensor0,transpose_b=True)  # (emb_dim,bs,feat_num,feat_num)

In [44]:
m

<tf.Tensor 'MatMul_2:0' shape=(5, 3, 4, 4) dtype=float32>

In [45]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [46]:
mm=sess.run(m)

In [51]:
o = tf.reshape(m,shape=[5,-1,4*4])
o = tf.transpose(o, perm=[1, 0, 2])  # [bs, emb_dim, H_1*m]
o

<tf.Tensor 'transpose_1:0' shape=(3, 5, 16) dtype=float32>

In [72]:
tf.split(0, 5 * [1], 2)

ValueError: Can't split scalars for 'split_6' (op: 'SplitV') with input shapes: [], [5], [].

In [42]:
sess.run(split_tensor0[0])

array([[[ 0.],
        [ 5.],
        [10.],
        [15.]],

       [[20.],
        [25.],
        [30.],
        [35.]],

       [[40.],
        [45.],
        [50.],
        [55.]]], dtype=float32)

# conv1d 

In [52]:
e = np.arange(3*4*25).reshape(3,4,5*5).astype(np.float32)  # [bs, emb_dim, f*f]  feat_num=m
emb = tf.Variable(e)  # [3,4,25]
f = np.arange(1*25*7).reshape(1,25,7).astype(np.float32)  # [1, feat_num*feat_num, H_k]  feat_num=m
fil = tf.Variable(f)  # [1,25,7]

In [53]:
conv = tf.nn.conv1d(emb,fil,stride=1,padding='VALID')  # [bs, emb_dim, H_k]

In [54]:
conv

<tf.Tensor 'conv1d/Squeeze:0' shape=(3, 4, 7) dtype=float32>

In [64]:
table = np.random.random([10,4])

In [70]:
    random_shifts = np.random.random(table.shape)
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    for i in range(10):
        table[i] /= table[i].sum()
        shifted_probabilities = random_shifts[i] - table[i]
        
        l = np.argpartition(shifted_probabilities, 1, axis=0)[0]
        table[i:,l] = 0
        result.append(l)

d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [71]:
result

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]